In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; }
.container {width:95% !important;}
</style>

In [ ]:
import numpy as np
%matplotlib notebook
from scipy import fftpack
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import SVG
from style import *

def color2bw(img):
    return np.dot(img, [0.299, 0.587, 0.114]) 

## Ancho de banda de un video

- ¿Cúanto pesa una imagen RGB de 1920x1080?

<center>1920 1080 3 size(uint8) = 49.766.400 b ~ 50 Mb</center>

- Tradicionalmente un video es una secuencia de imágenes a 24 cuadros por segundo (fps)
- ¿Cuánto ancho de banda se necesita para ver una película en tiempo real?

<center>50Mb  24 fps = 1200 Mb/s = 1.2 Gb/s</center>

- Este ancho de banda es infactible ¿Qué necesitamos para que un servicio de streaming pueda funcionar?

https://toolstud.io/video/filesize.php

# Compresión de datos

- Codificar la información usando "menos bits" que la representación original
- La compresión puede ser de tipo
    - *Lossless* (sin pérdidas): Los datos originales pueden reconstruirse perfectamente
    - *Lossy* (con pérdidas): Se reconstruye una versión aproximada de los datos originales


Recordemos el modelo de Shannon

<img src="images/shannon-diagram.svg" width="500">


Podemos hacer un modelo más detallado para el **transmisor** como

<img src="images/transmitter.svg" width="600">

donde


- **Transformación:** Cambia la representación de los datos para eliminar redundancia/correlaciones 
- **Cuantización:** Escoge un número fijo de valores representativos (diccionario), es decir acorta el "largo de palabra" de los datos
    - Largo de palabra: Cantidad de bits necesaria para representar un símbolo de código
    - Diccionario o alfabeto: Conjunto de símbolos o palabras
- **Codificación de fuente:** Convierte el diccionario fijo del cuantizador en un código de largo variable, que es más eficiente de transmitir
- **Codificación de canal:** Se encarga de "robustecer" el resultado anterior para que sobreviva la transmisión digital (parity, checksum)

## Compresión de imágenes

- Una imagen se puede proyectar al espacio de frecuencias sin pérdida de información
- En general vimos que la información se concentra en el centro del espectro
- Las tres redundancias:
    -  **Redundancia interpixel (transformación)** Alta correlación entre píxeles vecinos 
    - **Redundancia psicovisual (cuantización):** El ojo humano no puede resolver más de 32 niveles de grises:
    - **Redundancia de codificación:** Algunos tonos son más comunes que otros 
 

¿Cómo explotamos esto para reducir el tamaño de una imagen?

##  Ejemplo: Joint Photographic Experts Group (JPEG)

- Formato ampliamente usado para distribuir imágenes digitales
- Es un algoritmo de compresión con pérdidas para imágenes (lossy)
- Explota las siguientes características del sistema visual humano
    - Somos más sensibles a la iluminación que al color
    - Somos menos sensibles a los componentes de alta frecuencia
- Más componentes descartados: mayor compresión, y peor la calidad


Algoritmo:
1. Paso preliminar: RGB a YCbCr (y downsampling 4:2:2)
1. Transformación con **Discrete Cosine Transform** (DCT) en bloques de 8x8 sin traslape
1. Cuantización escalar
1. Codificación de Huffman


### Espacio YCbCr
- Y es la luminancia
- Cb y Cr son los componentes cromáticos azul y rojo, respectivamente

$$
Y = K_R R + K_G G + K_B B \\
C_b = 0.5 \frac{B - Y}{1 - K_B} \\
C_R = 0.5 \frac{R - Y}{1 - K_R}
$$
donde $K_R + K_G + K_B = 1$.


In [ ]:
# ITU-R BT.601 
dogo_color = plt.imread('images/lobo.jpg')
Y = color2bw(dogo_color)

plt.matshow(Y, cmap=plt.cm.Greys_r); plt.axis('off'); plt.title('Y')
C_b = np.dot(dogo_color, [-0.168736, -0.3312, 0.5])
plt.matshow(C_b, cmap=plt.cm.Greys_r); plt.axis('off'); plt.title('Cb')
C_r = np.dot(dogo_color, [0.5, -0.4186, -0.0813])
plt.matshow(C_r, cmap=plt.cm.Greys_r); plt.axis('off'); plt.title('Cr');

In [ ]:
plt.close(); fig, ax = plt.subplots(figsize=(9, 5), tight_layout=True)

def update(q_Y, q_C):
    ax.cla(); ax.axis('off')
    Y = np.round(np.dot(dogo_color, [0.299, 0.587, 0.114])*q_Y/255.0)*255.0/q_Y
    C_b = 128+np.round(np.dot(dogo_color, [-0.168736, -0.3312, 0.5])*q_C/255.0)*255.0/q_C
    C_r = 128+np.round(np.dot(dogo_color, [0.5, -0.4186, -0.0813])*q_C/255.0)*255.0/q_C
    rgb = np.zeros(shape=(Y.shape[0], Y.shape[1], 3))
    rgb[:,:,0] = Y + 1.402 * (C_r-128)
    rgb[:,:,1] = Y - .34414 * (C_b-128) - .71414 * (C_r-128)
    rgb[:,:,2] = Y + 1.772 * (C_b-128)
    np.putmask(rgb, rgb > 255, 255);
    np.putmask(rgb, rgb < 0, 0);    
    ax.imshow(np.uint8(rgb));
   
interact(update, q_Y=SelectionSlider_nice(options=[1, 2, 4, 8, 16, 32, 64, 128], value=128, description="Niveles de Y"),
         q_C=SelectionSlider_nice(options=[1, 2, 4, 8, 16, 32, 64, 128], value=128, description="Niveles de C"));

## Discrete Cosine Transform (DCT)

Sea una señal discreta y bidimensional $g[n_1, n_2]$ con índices $n_1 \in [0, N_1-1]$ y $n_2 \in [0, N_2-1]$ su DCT es 

$$
G_C[k_1, k_2] = \sum_{n_1=0}^{N_1-1} \sum_{n_2=0}^{N_2-1} 4 g[n_1, n_2] \cos \left ( \frac{\pi k_1}{2N_1}(2n_1+1)  \right) \cos \left ( \frac{\pi k_2}{2N_2}(2n_2+1)  \right) 
$$
y su inversa

$$
g[n_1, n_2] = \frac{1}{N_1 N_2}\sum_{k_1=0}^{N_1-1} \sum_{k_2=0}^{N_2-1} w[k_1]w[k_2]G[k_1, k_2] \cos \left ( \frac{\pi k_1}{2N_1}(2n_1+1)  \right) \cos \left ( \frac{\pi k_2}{2N_2}(2n_2+1)  \right), 
$$

donde 
$$
w[k] =\begin{cases}
1/2 & \text{ssi} ~~ k=0\\
1 & \text{ssi} ~~ k \neq 0
\end{cases} 
$$

La DCT bidimensional:
- es lineal, y cumple el principio de conservación de energía
- se puede descomponer en 2 aplicaciones de la DCT 1D
- es equivalente a la DFT de una señal "simetricamente extendida":

$$
y[k] =\begin{cases}
x[k] & \text{ssi} ~~ k<N\\
x[2N-1-k] & \text{ssi} ~~ N \leq k < 2N - 1 
\end{cases} 
$$
- Es decir que podemos usar el algoritmo FFT para calcular eficientemente la DCT

**Ojo:** La convolución en espacio original no es multiplicación en el espacio DCT


La base de la DCT:

In [ ]:
x = np.arange(0, 32, step=1)
X, Y = np.meshgrid(x, x)
fig, ax = plt.subplots(8, 8, figsize=(9, 8), tight_layout=False)
fig.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
for n in range(ax.shape[0]):
    for m in range(ax.shape[1]):
        cos_x = np.cos(np.pi*(2*X+1)*m/(2*len(x)))
        cos_y = np.cos(np.pi*(2*Y+1)*n/(2*len(x)))
        ax[n, m].matshow(cos_x*cos_y, cmap=plt.cm.RdBu_r, vmin=-1, vmax=1)
        ax[n, m].axis('off')


### Ejemplo: Transformación de una imagen usando DCT en bloques de 8x8

In [ ]:
img_seadoge = color2bw(plt.imread('images/lobo.jpg'))  
fig, ax = plt.subplots(figsize=(9, 6)); ax.axis('off')
ax.imshow(img_seadoge, cmap=plt.cm.Greys_r);

In [ ]:
DCT2 = lambda g, norm='ortho': fftpack.dct( fftpack.dct(g, axis=0, norm=norm), axis=1, norm=norm)
IDCT2 = lambda G, norm='ortho': fftpack.idct( fftpack.idct(G, axis=0, norm=norm), axis=1, norm=norm)

imsize = img_seadoge.shape
dct_matrix = np.zeros(shape=imsize)
dft_matrix = np.zeros(shape=imsize, dtype=np.complex128)

for i in range(0, imsize[0], 8):
    for j in range(0, imsize[1], 8):
        dct_matrix[i:(i+8),j:(j+8)] = DCT2( img_seadoge[i:(i+8),j:(j+8)] )
        dft_matrix[i:(i+8),j:(j+8)] = fftpack.fft2( img_seadoge[i:(i+8),j:(j+8)] )  


¿Cómo se ven uno a uno la DCT de los bloques?

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5), tight_layout=True)

def plot_values(ax, tile, fontsize=16):
    for i in range(8):
        for j in range(8):
            label = tile[i, j]
            ax.text(i, j, int(label), fontsize=fontsize, 
                    color='black', ha='center', va='center')
            
def update(block_idx=1):
    for ax_ in ax:
        ax_.cla(); ax_.axis('off')
    tile = img_seadoge[8*block_idx:8*block_idx+8, 8*block_idx:8*block_idx+8]
    ax[0].imshow(tile, cmap=plt.cm.Greys_r, 
                 vmin=img_seadoge.min(), vmax=img_seadoge.max())
    ax[0].set_title("%d 8x8 image block" %(block_idx)); 
    plot_values(ax[0], tile)
    tile = dct_matrix[8*block_idx:8*block_idx+8, 8*block_idx:8*block_idx+8]
    ax[1].imshow(tile, cmap=plt.cm.Greys_r, 
                 vmin=dct_matrix.min(), vmax=dct_matrix.max())
    ax[1].set_title("8x8 DCT")
    plot_values(ax[1], tile)

interact(update, block_idx=IntSlider_nice(min=0, max=100, value=0, 
                                          description="Bloque"));

¿Cómo se ven los bloques de la DCT si los concatenamos?

In [ ]:
plt.close(); 
fig, ax = plt.subplots(figsize=(10, 7), tight_layout=True)
ax.axis('off')
ax.matshow(dct_matrix, cmap=plt.cm.Greys_r, vmin=0, vmax=50);

Comparemos ahora la DFT y DCT en términos de reconstrucción con pérdidas

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 6), tight_layout=False)
fig.subplots_adjust(left=0.01, right=0.99, top=0.94, bottom=0.01)
def update(percent):
    for ax_ in ax.ravel():
        ax_.cla(); ax_.axis('off')
    imsize = img_seadoge.shape
    Npixels = imsize[0]*imsize[1]
    im_dct, im_dft = np.zeros(imsize), np.zeros(imsize)
    I = np.unravel_index(np.argsort(np.absolute(dct_matrix), axis=None)[::-1], 
                         dct_matrix.shape)
    dct_thresh = dct_matrix.copy()
    dct_thresh[I[0][int(Npixels*percent/100):], 
               I[1][int(Npixels*percent/100):]] = 0
    I = np.unravel_index(np.argsort(np.absolute(dft_matrix), axis=None)[::-1], 
                         dft_matrix.shape)
    dft_thresh = dft_matrix.copy()
    dft_thresh[I[0][int(Npixels*percent/100):], 
               I[1][int(Npixels*percent/100):]] = 0
    for i in range(0, imsize[0], 8):
        for j in range(0, imsize[1], 8):
            im_dct[i:(i+8),j:(j+8)] = IDCT2( dct_thresh[i:(i+8),j:(j+8)] )
            im_dft[i:(i+8),j:(j+8)] = np.real(fftpack.ifft2( dft_thresh[i:(i+8),j:(j+8)] ))
    
    ax[0, 0].imshow(im_dft, cmap=plt.cm.Greys_r); 
    ax[0, 0].set_title("DFT")
    ax[1, 0].imshow(im_dft[200:400, 250:600], cmap=plt.cm.Greys_r); 
    ax[0, 1].imshow(im_dct, cmap=plt.cm.Greys_r); 
    ax[0, 1].set_title("DCT")
    ax[1, 1].imshow(im_dct[200:400, 250:600], cmap=plt.cm.Greys_r); 
    
interact(update, percent=SelectionSlider_nice(options=[1., 2.5, 5., 7.5, 10., 20], value=20,
                                          description="Porcentaje de componentes retenidos"));

La DCT es equivalente a la DFT de la imagen simetricamente extendida:

In [ ]:
img_seadogo_sym = np.hstack((img_seadoge, np.fliplr(img_seadoge)))
img_seadogo_sym = np.vstack((np.flipud(img_seadogo_sym), img_seadogo_sym))
fig, ax = plt.subplots(figsize=(10, 6))
ax.axis('off')
ax.imshow(img_seadogo_sym, cmap=plt.cm.Greys_r);

- DCT y efectos de borde
- optimalidad en termines de aproximacion a transformacion KL
- optimalidad en terminos de señal markoviana con correlacion positiva

## Cuantización escalar

- Objetivo: Atacar la redundancia psicovisual
- Es una operación de redondeo/truncamiento
- Se define por: 
    - número de niveles $L$, 
    - fronteras de decisión $d_i$ 
    - valor de las representaciones $r_i$
$$
\begin{equation}
    Q(x)=
    \begin{cases}
      r_1, & d_0 < x \leq d_1 \\
      \vdots & \vdots \\
      r_i, & d_{i-1} < x \leq d_i \\
      \vdots & \vdots \\
      r_L, & d_{L-1} < x \leq d_L
    \end{cases}
\end{equation}
$$

### Cuantización uniforme

Sea un rango de valores en $[-V, V]$, una cuantización uniforme de $L$ niveles sería

$$
d_0 = -V, d_{L} = V
$$
La separación entre niveles es fija
$$
d_{i} = d_{i-1} + \Delta  = d_0 + i \Delta = - V + i \frac{2V}{L}
$$
y el valor de representación es el punto medio de cada nivel
$$
r_i = \frac{1}{2} (d_i + d_{i-1}) = -V + (2i-1) \frac{2V}{L}
$$ 
 


### Error de cuatización

Es la distancia entre el valor real y su versión cuantizada
$$
d_c = \|Q(x) -x\| 
$$

## Cuantización vectorial

Es la extensión del caso anterior a múltiples dimensiones

- Sea un set de datos N-dimensionales $X=\{x_1, x_2, \ldots, x_N\}$ 
- Se busca un set de prototipos $C=\{c_1, c_2, \ldots, x_L\}$ donde $L < N$
- La función de cuantización es $Q(x) = c_k$ donde $k = \text{arg}\min_i \|x - c_i\|$
- Hay que definir una métrica de distancia (*e.g* norma L2)

Algoritmo
1. Inicializar con prototipos aleatorios
2. Asignar los datos a su prototipo más cercano
3. Actualizar los prototipos como el valor medio de sus datos asignados
4. Iterar hasta converger

<center><img src="images/kmeans.gif" width="600"></center>

Si se usa norma L2 se obtiene el famoso algoritmo K-means

In [ ]:
from scipy.cluster.vq import vq
from sklearn.cluster import KMeans
img_color = plt.imread("images/atardecer.jpg")
img_pixels = np.reshape(img_color, (-1, 3))
kmeans = KMeans(n_clusters=10).fit(np.float32(img_pixels)) 
centroids = kmeans.cluster_centers_
qnt,_ = vq(np.float32(img_pixels), centroids)
centers_idx = np.reshape(qnt, (img_color.shape[0], img_color.shape[1]))
clustered = np.uint8(centroids[centers_idx])

fig, ax = plt.subplots(1, 2, figsize=(9, 4), tight_layout=False)
fig.subplots_adjust(left=0.01, right=0.99, top=0.99, bottom=0.01)
ax[0].imshow(img_color); ax[0].axis('off');
ax[1].imshow(clustered); ax[1].axis('off');

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=((6, 5)))
ax = fig.gca(projection='3d')
ax.scatter(centroids[:, 0], centroids[:, 1], centroids[:, 2], 
           c=centroids/255., s=500);
ax.set_xlim([0, 255]); ax.set_ylim([0, 255]); ax.set_zlim([0, 255]);

### Cuantización en JPEG

- Se cuantizan los bloques de 8x8 componentes DCT 
- El nivel de los componentes se redondea según una matriz de cuantización Q
- Q fue diseñada tal que componentes de alta frecuencia se cuantizan en menos niveles
- El bloque cuantizado se obtiene como $\text{ROUND}\left(\frac{G_C}{Q}\right)$

In [ ]:
Q = np.array([[16, 11, 10, 16, 24, 40, 51, 61],[12, 12, 14, 19, 26, 58, 60, 55],
              [14, 13, 16, 24, 40, 57, 69, 56],[14, 17, 22, 29, 51, 87, 80, 62],
              [18, 22, 37, 56, 68, 109, 103, 77],[24, 35, 55, 64, 81, 104, 113, 92],
              [49, 64, 78, 87, 103, 121, 120, 101],[72, 92, 95, 98, 112, 100, 103, 99]])


fig, ax = plt.subplots(1, 3, figsize=(10, 4), tight_layout=True)
def update(block_idx=1):
    for ax_ in ax:
        ax_.cla(); ax_.axis('off')
    tile = img_seadoge[8*block_idx:8*block_idx+8, 8*block_idx:8*block_idx+8]
    ax[0].imshow(tile, cmap=plt.cm.Greys_r)
    ax[0].set_title("8x8 %d'th block" %(block_idx)); 
    plot_values(ax[0], tile, fontsize=12)
    tile = dct_matrix[8*block_idx:8*block_idx+8, 8*block_idx:8*block_idx+8]
    ax[1].imshow(tile, cmap=plt.cm.Greys_r)
    ax[1].set_title("8x8 DCT\n %d nonzero" %(np.count_nonzero(tile))); 
    plot_values(ax[1], tile, fontsize=12)
    quant = np.round(tile/Q)
    ax[2].imshow(quant, cmap=plt.cm.Greys_r)
    ax[2].set_title("8x8 Quantized\n%d nonzero" %(np.count_nonzero(quant))); 
    plot_values(ax[2], quant, fontsize=12)

interact(update, block_idx=IntSlider_nice(min=0, max=100, value=0, description="Block tile"));

In [ ]:
area = img_seadoge.shape[0]*img_seadoge.shape[1]
fig, ax = plt.subplots(1, 2, figsize=(10, 4), tight_layout=False)
fig.subplots_adjust(left=0.01, right=0.99, top=0.94, bottom=0.01)

def update(percent):
    for ax_ in ax.ravel():
        ax_.cla(); ax_.axis('off')
    im_dct = np.zeros(imsize)
    nnz = np.zeros(dct_matrix.shape)
    if (percent < 50):
        S = 5000/percent
    else:
        S = 200 - 2*percent 
    Q_dyn = np.floor((S*Q + 50) / 100);
    Q_dyn[Q_dyn == 0] = 1
    for i in range(0, imsize[0], 8):
        for j in range(0, imsize[1], 8):
            quant = np.round(dct_matrix[i:(i+8),j:(j+8)]/Q_dyn) 
            im_dct[i:(i+8),j:(j+8)] = IDCT2(quant)
            nnz[i, j] = np.count_nonzero(quant)
    
    ax[0].imshow(img_seadoge, cmap=plt.cm.Greys_r); 
    ax[0].set_title("%0.2f MB" %(area*8/1e+6))
    ax[1].imshow(im_dct, cmap=plt.cm.Greys_r); 
    ax[1].set_title("%0.2f MB" %(np.sum(nnz)*8/1e+6))
interact(update, percent=FloatSlider_nice(min=0, max=100, step=0.01, value=100, 
                                     description="Nivel de compresión"));

# Codificación de fuente

- Es el proceso que asigna un código a cada mensaje
- El largo de las "palabras de código" puede ser **fijo** o **variable**
- **Ejemplo:** Queremos codificar las letras del alfabeto (27) usando dos símbolos (código binario)
    - Para largo fijo necesitamos al menos 5 bits ($2^5 = 32$)
- Pero algunas letras se ocupan más que otras (*e.g.* vocales) 
- ¿Podemos aprovechar esto para comprimir un mensaje?

In [ ]:
import string
from collections import Counter

with open('quijote.txt', 'r') as file:
    texto = file.read()
print(texto)
texto = texto.translate({ord(k): None for k in string.digits})
for symbol in string.punctuation:
    texto = texto.replace(symbol, "")
texto = texto.replace(" ", "").replace("\n", "")
texto = texto.lower().encode('ascii', 'ignore').decode("utf-8")
print("\nLargo del texto: {0}, Cantidad de bits: {1}".format(len(texto), len(texto)*5))
Counter(texto).most_common()

**Intuición:** Podríamos reducir la cantidad de bits si usamos códigos más cortos para las letras más frecuentes

***

# Teoría de la información

- Estudio de la cuantificación y transmisión de la información 
- Propuesto por **Claude Shannon** en 1948: *A Mathematical Theory of Communication*
- Proporciona herramientas para medir la cantidad de información 
- Aplicaciones en telecomunicaciones, computación y biología (genética)
- Podemos usar estas herramientas para evaluar los resultados de la compresión

### Información y entropía

- Sea una fuente $X$ que emite $M$ mensajes 
- Cada mesaje tiene con probabilidad de ocurrencia $p_i \in [0, 1]$ y $\sum_{i=1}^M p_i = 1$
- La cantidad de información de los mensajes es $\log_2 \left( \frac{1}{p_i} \right)$ y se mide en **bits** 
- La cantidad de información promedio de la fuente es
$$
\begin{align}
H (X) &= \mathbb{E} \left [ - \log P(X) \right ]  \nonumber \\
&= - \sum_{i=1}^M P(x_i) \log_2 P(x_i)  \nonumber \\
&= - \sum_{i=1}^M p_i \log_2 p_i  \quad \text{[bits]} \nonumber
\end{align}
$$
que se conoce como **Entropía de Shannon** 

- La entropía es siempre positiva y acotada $0 \leq H(X) \leq - \log_2(1/M)$

### Ejemplo: Moneda con truco

- Sea una variable aleatoria $X$ que modela el resultado de lanzar una moneda
- Asumamos que el resultado puede tomar solo dos valores: Cara $o$ o Cruz $x$
- La probabilidad de que salga cara es $p_o = p$
- La probabilidad de que salga cruz es $p_x = 1- p$
- Luego la entropía es 

$$
\begin{align}
H(X) &= -\sum_{i=1}^2 p_i \log_2 p_i \nonumber \\ 
&= -p_x \log (p_x) - p_o \log p_o \nonumber \\
&= - p \log(p) - (1-p) \log(1-p)
\end{align}
$$


- ¿En que casos la entropía es mínima? ¿En qué caso es máxima?
- ¿Puedes relacionar la entropía con la aleatoridad de la moneda?


Ojo: $\lim_{z\to 0^+} z \log 1/z = 0$

In [ ]:
p = np.linspace(0, 1, num=100)
H = -p*np.log2(p+1e-10) - (1-p)*np.log(1-p + 1e-10)
fig, ax = plt.subplots(2, figsize=(6, 6), sharex=True)
ax[1].set_xlabel('p')
ax[0].set_title('Información del evento cara')
ax[0].plot(p, -np.log2(p+1e-2))
ax[1].set_title('Entropía de la moneda')
ax[1].plot(p, H);

### Ejemplo: Meteorólogos del siglo XIX

- Nos encontramos a finales del siglo XIX
- La estación meteorológica de Niebla hace una predicción del tiempo en Valdivia
- Esta información se envía a Valdivia a través de telegrafo
- Indique la cantidad de información promedio que envía la estación a Valdivia en cada escenario

**Escenario 1:** Dos posibilidades: Lluvia y nublado, con probabilidad $1/2$ y $1/2$, respectivamente

**Escenario 2:** Una posibilidad: Lluvia, con probabilidad $1$

**Escenario 3:** Cuatro posibilidades: Lluvia, Nublado, Nubosidad parcial, soleado, con probabilidad $1/2$, $1/4$, $1/8$ y $1/8$, respectivamente


>Un bit es la cantidad de información que se requiere para escoger entre **dos** alternativas equiprobables

#### Escenario 3

1. Las probabilidades de cada mensaje son $2^{-1}$, $2^{-2}$, $2^{-3}$ y $2^{-3}$
1. La cantidad de información de cada mensaje es: 1, 2, 3 y 3 bits, respectivamente
1. La entropía es $1/2 + 1/2 + 3/8 + 3/8 = 1.75$ bits

Consideremos el alfabeto de códigos construido con {0, 1}

¿Cómo le asignamos un código a cada alternativa?


### Código de ancho fijo

- Tenemos cuatro estados, necesitamos 2 bits
- Cada estado: 00, 01, 10, 11
- En este caso resulta equivalente a asumir equiprobabilidad 
- La entropía es 2 bits

### Código de ancho variable (prefijo)

- Se usa 1, 2, 3 y 3 bits para cada estado, según su probabilidad de aparición
- La entropía es 1.75 bits
- Podemos describir este escenario según
    - Primera decisión equiprobable: Lluvia **(0)** vs El resto (1)
    - Segunda decisión equiprobable: Nublado **(10)** vs El resto menos lluvia (11)
    - Tercera decisión equiprobable: Nubosidad parcial **(110)** vs soleado **(111)**
- Podemos escribir esto como un dendograma

<center><img src="images/dendogram.png" width="600"></center>

- Algoritmo de codificación con forma de árbol en base 2
- Los mensajes codificados están en las hojas del árbol
- **Código préfijo**: Ningún código puede ser prefijo de otro. 
- El código prefijo garantiza decodificación sin ambiguedad


Decodifique la predicción del tiempo para los próximos tres días: 101100 

Código ambiguo: Si el código de lluvia fuera **1**, decodifique el siguiente mensaje: 11111

### Ejemplo: Entropía y cantidad de bits del fragmento del famoso texto


In [ ]:
# Código de largo fijo:
print(5*len(texto))
# Código de largo variable:
freq = np.array(list(Counter(texto).values()))
p = freq/np.sum(freq)
print(int(-np.sum(p*np.log2(p))*len(texto)))

**Reflexionemos:** 
- ¿Es la codificación de largo variable *lossless* o *lossy*?
- En ciertos casos las palabras son más largas de lo que eran originalmente, ¿Cómo comprimimos entonces?

## Codificación de Huffman

Un algoritmo sencillo de codificación de tipo prefijo:

1. Se estima la probabilidad $p_i$ de cada símbolo
1. Se ordenan los símbolos en orden descendente según $p_i$
1. Juntar los dos con probabilidad menor en un grupo, su probabilidad se suma
1. Volver al paso 2 hasta que queden dos grupos
1. Asignarle 0 y 1 a las ramas izquierda y derecha del árbol, respectivamente

<center><a href="http://www.skylondaworks.com/sc_huff.htm"><img src="images/huff.gif" width="600"></a></center>


**Debilidad de Huffman:** 
- Códigos con diccionarios/probabilidades variables
- En ese caso combiene usar codificación aritmética o Lempel-Ziv



***
Hasta aquí nomas

Actualizar de aquí para abajo:

## Teorema de codificación de fuente (source coding theorem)

El largo de palabra promedio $\widehat L$ de un algoritmo de codificación binario prefijo satisface
$$
H(X) \leq \widehat L \leq H(X) + 1
$$
Este teorema justifica la definición de entropía como meduda de la cantidad de información/incerteza

## Teorema de Shannon-Hartley

- Sea un canal con ancho de banda B [Hz] (rango de frecuencias que un canal puede transmitir) y potencia de señal S [W] y potencia del ruido N [W] (aditivo blanco gaussiano), entonces su capacidad es
$$
C = B \log_2 \left(1 + \frac{S}{N} \right) \text{[bits/s]}
$$
- Si la velocidad de transmisión de un canal es R [bits/s] y R < C entonces la probabilidad de errores de comunicación tiende a cero.
- Las limitaciones de un sistema de comunicación son **Ancho de banda** y el ruido **Ruido**

## H.264/MPEG-4

- Aplicación de la DCT para comprimir a través de varios frames

https://pywavelets.readthedocs.io/en/latest/